In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1580876/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_1580876/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_2_3_2,0.836662,0.212389,-0.810434,0.855176,0.222548,1.578915,1.335037,0.094570,1.716855e+14,0.471750,0.662786,0.479179,69.005229,140.760008,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1,model_2_2_7,0.809217,0.205001,-0.822939,0.770128,0.259941,1.593725,1.344259,0.236230,1.852926e+14,0.509844,0.606126,0.517874,68.694599,140.449379,"Hidden Size=[8], regularizer=0.02, learning_ra..."
2,model_2_3_6,0.853762,0.200659,-0.902628,0.875361,0.199249,1.602431,1.403022,0.081389,1.646600e+14,0.446373,0.698089,0.453403,69.226398,140.981178,"Hidden Size=[8], regularizer=0.02, learning_ra..."
3,model_2_3_8,0.867592,0.172767,-0.990446,0.882730,0.180405,1.658345,1.467781,0.076577,1.590112e+14,0.424742,0.726642,0.431431,69.425098,141.179878,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4,model_2_1_1,0.716917,0.158060,-0.880010,0.734952,0.385700,1.687827,1.386344,0.297191,2.163377e+14,0.621047,0.415571,0.630828,67.905391,139.660171,"Hidden Size=[8], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,model_3_18_9,0.766404,-0.328530,-2.879660,-0.122304,0.318274,2.663291,2.860911,1.428117,3.075986e+13,0.564157,0.003325,0.573042,100.289688,206.834664,"Hidden Size=[12], regularizer=0.02, learning_r..."
466,model_3_2_3,0.761340,-0.350456,-2.411234,0.778234,0.325174,2.707245,2.515489,0.475952,1.341389e+14,0.570240,-0.018283,0.579221,100.246790,206.791766,"Hidden Size=[12], regularizer=0.02, learning_r..."
467,model_3_1_0,0.636826,-0.368784,-2.616879,0.474886,0.494823,2.743987,2.667134,0.297538,1.801505e+14,0.703437,-0.549541,0.714515,99.407110,205.952086,"Hidden Size=[12], regularizer=0.02, learning_r..."
468,model_3_20_9,0.933206,-0.560778,-1.238106,0.871857,0.091007,3.128876,1.650408,0.311849,7.254137e+13,0.301673,0.715012,0.306424,102.793642,209.338618,"Hidden Size=[12], regularizer=0.02, learning_r..."
